# Proxy checker

In [ ]:
with open("proxies.csv", "r") as f:
    proxies = f.read().split()
    
print(len(proxies))
print(len(set(proxies)))

with open("proxies.csv", "w") as f:
    f.write("\n".join(sorted(set(proxies))))

In [8]:
from time import sleep
import requests
from lxml import html
import csv

with open("approved_proxies.csv", "r") as f:
    proxies = f.read().split()

with open("proxies_https.csv", "w") as f:    
    for proxy in proxies:        
        url = "https://www.google.com/finance/getprices?i=60&p=10d&f=d,o,h,l,c,v&df=cpct&q=AAPL"
        try:
            page = requests.get(url, timeout=6, proxies={'http': proxy, 'https': proxy})
        except Exception as e:
            pass
        else:    
            lines = list(csv.reader(page.content.decode('utf-8').splitlines()))
            if lines[3][0] != "INTERVAL=60":
                print(page.content.decode('utf-8')[:100])
            else:
                f.write("{}\n".format(proxy))
                print(proxy)

104.237.246.26:3128
103.14.8.239:8080
103.14.8.239:80
115.124.73.122:80
138.197.12.124:3128
138.197.12.124:8080
138.197.79.173:3128
138.197.32.85:80
138.197.79.173:8799
138.197.32.85:8080
138.197.79.173:80
122.116.165.159:3128
138.255.105.55:8080
144.217.133.42:8080
144.217.31.225:8080
144.217.31.225:3128
144.217.31.225:80
138.201.63.123:31288
144.217.96.81:8080
144.217.96.81:3128
149.56.81.59:8080
152.194.72.134:8080
124.244.39.44:3128
151.252.120.177:8080
158.69.78.208:3128
158.69.78.208:8080
158.69.78.208:80
158.69.78.208:8799
163.121.188.2:8080
163.121.188.3:8080
167.114.82.173:80
167.114.82.173:3128
176.159.230.207:3128
168.187.239.10:3128
144.76.106.53:80
177.54.144.218:3128
177.54.144.218:8799
162.243.251.200:8080
177.67.86.121:8080
177.67.84.135:8080
185.107.80.44:3128
185.28.193.95:8080
176.126.245.23:3128
177.67.81.248:8799
177.67.81.248:8080
176.31.185.36:8080
177.67.81.248:3128
177.135.90.139:3128
178.33.4.48:3128
189.111.250.209:3128
192.208.184.133:80
192.169.144.40:8080


In [ ]:
import requests
from lxml import html
import queue
import threading
    
NUM_THREADS = 300

def check_proxies():   
    
    tasks = queue.Queue() 
    responses = queue.Queue() 
    
    with open("proxies.csv", "r") as f:
        proxies = f.read().split()
    
    for p in proxies:
        tasks.put(p)
    
    def worker():
        while True:
            proxy = tasks.get()
            if proxy is None:
                break 
                
            try:
                base_url = "http://www.nasdaq.com/symbol/{symbol}/time-sales?time={time}&pageno={pageno}"
                url = base_url.format(symbol="AAPL", time=1, pageno=1)
                page_res = requests.get(url, timeout=6, proxies={'http': "http://{}".format(proxy)})
                tree = html.fromstring(page_res.content)
            except Exception as e:
                pass
            else:
                try:
                    select = tree.find('.//select[@id="quotes_content_left_DropDowntimerange"]')
                    if select is None or int(select.value) != 1:
                        pass
                    else:        
                        tree = html.fromstring(page_res.content)
                        table = tree.find('.//table[@id="AfterHoursPagingContents_Table"]')
                        if table is None:
                            pass
                        else:
                            responses.put(proxy)
                except Exception:
                    print(e)
                 
            tasks.task_done()
    

    threads = []
    for i in range(NUM_THREADS):
        t = threading.Thread(target=worker)
        t.start()
        threads.append(t)

    # block until all tasks are done
    tasks.join()

    # stop workers
    for i in range(NUM_THREADS):
        tasks.put(None)

    for t in threads:
        t.join()
        
    approved = []
    while True:
        try:
            proxy = responses.get(block=False)
        except queue.Empty:
            break
        else:
            approved.append(proxy)
            
    print(len(approved))
    with open("approved_proxies.csv", "w") as f:
        f.write("\n".join(approved))

check_proxies()